## 1. Prepare the data

In [2]:
# Get latest Wikipedia English dump (this will take more than 4 hours)
# ! wget "http://download.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2"

In [3]:
# Extract text using WikiExtractor (this will take about 3 hours)
# ! python -m wikiextractor.WikiExtractor -o "data/wikipedia/" --json \
# --filter_disambig_page \
# --processes 8 \
# "data/enwiki-latest-pages-articles.xml.bz2"

## 2. Index wikipedia

In [1]:
HOST = 'localhost' 
PORT = 9200 
INDEX_NAME = 'wikipedia_100_stride_50'

from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host=HOST, port=PORT, username="", password="", index=INDEX_NAME)

09/18/2020 17:46:48 - INFO - elasticsearch -   PUT http://localhost:9200/wikipedia_100_stride_50 [status:400 request:0.004s]


In [ ]:
# clear existing index (optional)
if document_store.client.indices.exists(index=document_store.index):
    print('clear existing inddex')
    document_store.client.indices.delete(index=document_store.index)

In [ ]:
# Get all dirs in wikipedia folder
from os import listdir
from os.path import isfile, join
import json
from tqdm import tqdm

wikidata_path = "../data/wikipedia"
onlydirs = [f for f in listdir(wikidata_path) if not isfile(join(wikidata_path, f))]

dicts = []
bulk_size = 5000

pbar = tqdm(onlydirs)
for directory in pbar:
    subdirs = [f for f in listdir(join(wikidata_path,directory)) if not isfile(join(wikidata_path,directory))]
    pbar.set_description(f"Processing wikipedia folder {directory}")

    for file in subdirs:
        f = open(join(wikidata_path,directory,file), "r") 
        
        # Each text file contains json structures separated by EOL
        articles = f.read().split("\n")
        
        for article in articles:
            if len(article)==0: continue

            # Article in json format
            json_formatted_article = json.loads(article)

            # Rename keys
            document = {"id": json_formatted_article["id"],
                        "name": json_formatted_article["title"],
                        "url": json_formatted_article["url"], 
                        "text": json_formatted_article["text"]}

            # Add document to bulk
            dicts.append(document)
            
            if len(dicts)>=bulk_size:
                # Index bulk
                try:
                    document_store.write_documents(dicts)
                    dicts.clear()
                except:
                    print("Bulk not indexed")
        
    
if len(dicts) > 0:
    print('final round')
    document_store.write_documents(dicts)
            
print('finished')

## 3. QA on wikipedia

In [2]:
HOST = 'localhost' 
PORT = 9200 
INDEX_NAME = 'wikipedia_100_stride_50'

from haystack import Finder
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host=HOST, port=PORT, username="", password="", index=INDEX_NAME)

09/18/2020 17:47:08 - INFO - elasticsearch -   PUT http://localhost:9200/wikipedia_100_stride_50 [status:400 request:0.004s]


In [3]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)
READER_DiR = "../models/roberta-base-squad2"
READER_DiR = "../models/electra-base-squad2"
reader = TransformersReader(model=READER_DiR, tokenizer=READER_DiR,  use_gpu=0)

In [4]:
import logging
logging.disable(logging.WARNING)
finder = Finder(reader, retriever)
prediction = finder.get_answers(question="who is the father of Arya Stark", 
                                top_k_retriever=10, 
                                top_k_reader=3)

In [5]:
print_answers(prediction)

{   'answers': [   {   'answer': 'Lord Eddard Stark,',
                       'context': 'the Hand to honour her father Lord Eddard '
                                  'Stark, Sansa Stark is enchanted by t',
                       'document_id': 'sBXkiHQBc6r1xEMRXkLv',
                       'meta': {   'id': '1183383',
                                   'name': 'Sansa Stark',
                                   'segment_id': 10,
                                   'url': 'https://en.wikipedia.org/wiki?curid=1183383'},
                       'offset_end': 504,
                       'offset_start': 486,
                       'probability': 0.9655782551513105,
                       'score': None},
                   {   'answer': 'King Robert.',
                       'context': 'ey lacks a father figure from King Robert. '
                                  'While escorting Sansa home, h',
                       'document_id': 'KahJiXQBc6r1xEMR94mw',
                       'meta': {   '

In [9]:
docs = retriever.retrieve('who is the father of Arya Stark', top_k=3)

In [10]:
docs[0]

Document(id='sBXkiHQBc6r1xEMRXkLv', text="Sansa are walking through the woods, Joffrey notices Arya sparring with the butcher's boy, Mycah. A fight breaks out and Joffrey is attacked by Nymeria (Arya's direwolf) after Joffrey threatens to hurt Arya. Sansa lies to King Robert about the circumstances of the fight in order to protect both Joffrey and her sister Arya. Since Arya ran off with her wolf to save it, Sansa's wolf is killed instead, estranging the Stark daughters. During the Tourney of the Hand to honour her father Lord Eddard Stark, Sansa Stark is enchanted by the knights performing in the event. At the request", external_source_id=None, question=None, query_score=32.162544, meta={'id': '1183383', 'url': 'https://en.wikipedia.org/wiki?curid=1183383', 'segment_id': 10, 'name': 'Sansa Stark'}, tags=None)